Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import time
from dotenv import load_dotenv
import logging as log
# from matplotlib import pyplot as plt
# from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
from stockanalyzer import analyze_stock, stock_data_setup
import importlib

In [2]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

# Initialize an empty list to store the stocks
stocks_list = []

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

### Stocks Data

In [3]:
ticker = ['MRNA']

client = OpenAI()

file_stocks = stock_data_setup(client=client, ticker=ticker, type='price', dic_files=dic_files)
file_cashflow = stock_data_setup(client=client, ticker=ticker, type='cash', dic_files=dic_files)
file_income_stmt = stock_data_setup(client=client, ticker=ticker, type='income', dic_files=dic_files)

# Have to manually update the tool_resources because the file_id can change
dic_assistants['fin_analyst']['tool_resources'] = {
	'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
}

with open('config/assistants.json', 'w') as json_file:
	json.dump(dic_assistants, json_file)
	print(f'assistants.json file has been updated')

fin_analyst = AgentHandler(
	client = client, 
	new=False,
	assistant_name = 'fin_analyst',
	dic_file = dic_assistants
	)

fin_consultant = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'fin_consultant',
	dic_file = dic_assistants
	)

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_stocks.csv, file id: file-YBzOQXrOH9LqL8EDB0iYwyWG has been deleted
file name: df_stocks.csv is uploaded, new file id: file-jWdAWCIZ5J7BJ99WXAdcsIP5
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_cashflow.csv, file id: file-fNBgk4EQZvu3WpgQhCToc8NE has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-RevxbBVeaDqej9IbsegsN3V1
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_income_stmt.csv, file id: file-KEBS1yE5aC3JNrDfOhiPmcGe has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-1kT11cyKtcLCE1068PZ6zYHj
openai_upload_files/openai_files.json file has been updated
assistants.json file has been updated
Assistant has been updated, name: fin_analyst, id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
Assistant has been updated, name: fin_consultant, id: asst_mPlBQ4ZF8YIF238DWqWRhV4U


In [4]:
prompt_start = f"This is the financial consultant.\
	The client wants advice on the following stocks.\
	I need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\
	You will be provided with the data of these stocks in .csv format. Use them as you see fit.\
	Stocks: {ticker}"

thread = ThreadManager(
	client=client,
	prompt=prompt_start
)

thread.run_thread(
	assistant=fin_analyst
	# prompt=prompt_start,
	# attachments=[file_stocks.file_id, file_cashflow.file_id, file_income_stmt.file_id]
)

next_prompt = thread.last_message

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > code_interpreter

import pandas as pd

# Load the data from the uploaded CSV file
file_path = '/mnt/data/file-jWdAWCIZ5J7BJ99WXAdcsIP5'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
data.head()
assistant > Here are the information gathered from the financial analyst: 

These are the stocks the client wants advice on: 
- Moderna, Inc. (MRNA)

Based on the provided data, here is an analysis of Moderna, Inc. (MRNA):

### Historical Price Analysis (Duration: December 2018 to Present)

1. **Price Trends**:
   - **Initial Opening Price**: $22.00 (Dec 7, 2018)
   - **Recent Closing Price**: $18.76 (Dec 13, 2018)
   - **Historical High and Low**:
     - **Highest Price**: This can be analyzed by looking at the "High" column across all records.
     - **Lowest Price**: This can be analyzed by looking at the "Low" column across all records.

2. **Volatility**:
   - Significant fluctuations were observed within the initial da

In [5]:
next_prompt = 'This message is for testing. Can you give a short sample response.'

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)

print(thread.last_message)


assistant > Sure, here's a short sample response:

---

### Recommendation for Moderna, Inc. (MRNA)

**Strengths**:
1. **High Growth Potential**: Strong involvement in mRNA technology.
2. **Significant Market Interest**: High trading volumes indicate investor confidence.

**Weaknesses**:
1. **Volatility**: Significant price fluctuations present higher risk.
2. **No Dividends**: Not suitable for income-focused investors.

**Fit with Strategy**:
- **Growth-Oriented Investors**: Suitable due to high growth potential.
- **Risk-Averse Investors**: Caution advised due to volatility.
- **Income-Focused Investors**: Not recommended since it doesn’t offer dividends.

### Final Recommendation:
Moderna, Inc. (MRNA) is recommended for growth-oriented investors with higher risk tolerance. Not ideal for conservative or income-focused strategies.

---

Please let me know if you need any further details!get_last_message initiated
asst_id = blank
message.assistant_id != asst_id, record previous messag

In [6]:
thread.df_messages

,message_id,assistant_id,created_at,file_ids,role,run_id,message_text,_msg_loc
0,msg_ix1bKCixCKmonpOUUcwEA1ma,None,1727333036,[],user,None,This is the financial consultant.\tThe client ...,0
1,msg_av7UCA1kgMGRZ01gCpv4Ury6,asst_Mqf9cO1sDTOd4UMYwcFfIQrA,1727333042,[],assistant,run_E0zS7pImG3gCfk5W0euE7UG2,Here are the information gathered from the fin...,1
2,msg_MV1Z4VnNef62FVL3JloElTIk,None,1727333054,[],user,None,Here are the information gathered from the fin...,2
3,msg_lSPZOxGoeUugyaV1yrokW6Ey,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1727333055,[],assistant,run_XvEl5sGLSsT7me9d5BFLU4d8,"### Recommendation for Moderna, Inc. (MRNA)\n\...",3
4,msg_YtXAmLTidZAAWuOaWWswUtLE,None,1727333074,[],user,None,This message is for testing. Can you give a sh...,4
5,msg_RVGjKzoJzEA545GR4csDTgKz,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1727333075,[],assistant,run_KHvUeX1f2U5WBrtgA2BwTDoc,"Sure, here's a short sample response:\n\n---\n...",5


In [35]:
thread.df_messages

,message_id,assistant_id,created_at,file_ids,role,run_id,message_text,_msg_loc
0,msg_jNKhekiTCZtm8w4FPvX15gAj,None,1726556528,[],user,None,This is the financial consultant.\tThe client ...,0
1,msg_ZnRQdMDKHoD1H5rtlXqHchrD,asst_Mqf9cO1sDTOd4UMYwcFfIQrA,1726556533,[],assistant,run_SUXZgjKFGQXWadJINBkRKVKT,Here are the information gathered from the fin...,1
2,msg_Uk5Nu4x58DtDkKMAq2hD1caL,None,1726556545,[],user,None,Here are the information gathered from the fin...,2
3,msg_OeTIVYGnagoJeGndj95UJJFG,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1726556546,[],assistant,run_e3Wu9olqlU7Ts7zIrWRpeAoY,### Recommendation for Moderna Inc. (MRNA)\n\n...,3
4,msg_2oqodrxs428y1uKMA8I38ikI,None,1726556574,[],user,None,This message is for testing. Can you give a sh...,4
5,msg_tXf7w2DLQf3jXx2Vihici8De,asst_mPlBQ4ZF8YIF238DWqWRhV4U,1726556575,[],assistant,run_zhpjoi5bMscVc2i6fZWGL35t,### Recommendation for Moderna Inc. (MRNA)\n\n...,5


In [20]:
thread.delete_thread()

thread: thread_mPIjTp06Kua0Y9HZvV0eUdh3 has been deleted.


In [40]:
tick = yf.Tickers('MSFT')

In [53]:
tick.tickers['MSFT'].quarterly_balance_sheet.T.reset_index()

,index,Ordinary Shares Number,Share Issued,Net Debt,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,Capital Lease Obligations,...,Raw Materials,Receivables,Accounts Receivable,Allowance For Doubtful Accounts Receivable,Gross Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Other Short Term Investments,Cash And Cash Equivalents,Cash Equivalents,Cash Financial
0,2024-06-30,7434000000.0,7434000000.0,33315000000.0,67127000000.0,121660000000.0,320107000000.0,34448000000.0,121660000000.0,15497000000.0,...,394000000.0,56924000000.0,56924000000.0,-830000000.0,57754000000.0,75531000000.0,57216000000.0,18315000000.0,6744000000.0,11571000000.0
1,2024-03-31,7432305794.0,7432305794.0,45808000000.0,79911000000.0,105161000000.0,318594000000.0,28655000000.0,105161000000.0,14469000000.0,...,341000000.0,44029000000.0,44029000000.0,-616000000.0,44645000000.0,80013000000.0,60379000000.0,19634000000.0,8053000000.0,11581000000.0
2,2023-12-31,7431612107.0,7431612107.0,56914000000.0,88374000000.0,89441000000.0,312487000000.0,26377000000.0,89441000000.0,14155000000.0,...,382000000.0,42831000000.0,42831000000.0,-591000000.0,43422000000.0,80982000000.0,63677000000.0,17305000000.0,7769000000.0,9536000000.0
3,2023-09-30,7431000000.0,7431000000.0,NaN,84989000000.0,144029000000.0,292216000000.0,82794000000.0,144029000000.0,13487000000.0,...,520000000.0,36953000000.0,36953000000.0,-512000000.0,37465000000.0,143945000000.0,63493000000.0,80452000000.0,73074000000.0,7378000000.0
4,2023-06-30,7432000000.0,7432000000.0,12533000000.0,59965000000.0,128971000000.0,253460000000.0,80108000000.0,128971000000.0,12728000000.0,...,709000000.0,48688000000.0,48688000000.0,-650000000.0,49338000000.0,111256000000.0,76552000000.0,34704000000.0,26226000000.0,8478000000.0
5,2023-03-31,NaN,NaN,21648000000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# from selenium import webdriver

driver = webdriver.Chrome()

# Define the URL for the stock news (MSFT in this case)
url = "https://ca.finance.yahoo.com/quote/MSFT/news/"

driver.get(url)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

# Function to simulate infinite scrolling and grab more news links
def infinite_scroll(driver, max_links=100):
    news_links = set()
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    i = 0

    while len(news_links) < max_links:
        print(i)
        print(news_links)
        # Scroll down the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait for the page to load
        time.sleep(5)
        
        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Find all article elements that contain the news links
        articles = soup.find_all('a', href=True)
        
        for article in articles:
            href = article['href']
            if "/news/" in href:  # Ensure it's a news link
                full_url = f"https://ca.finance.yahoo.com{href}"
                news_links.add(full_url)
                
            # If we reach the desired number of links, stop scrolling
            if len(news_links) >= max_links:
                print(len(news_links))
                break
        
        # Calculate new scroll height to determine if we've reached the bottom
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print(f'new_height check, length of news_links:{len(news_links)}')
            break  # No more content to load
        
        last_height = new_height

        i+=1
    
    return list(news_links)

# Scrape the first 100 news links
news_links = infinite_scroll(driver, max_links=100)


In [7]:
from selenium.webdriver.common.action_chains import ActionChains

def natural_scroll_and_parse(driver, pause_time, scroll_limit, scroll_amount=1000):
    news_links = []

    for _ in range(scroll_limit):
        print(f'loop:{_}')
        # Scroll down in smaller steps
        ActionChains(driver).scroll_by_amount(0, scroll_amount).perform()
        
        # Wait for new content to load
        time.sleep(pause_time)

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Find all news links on the page
        articles = soup.find_all('a', href=True)  # Update class name as needed
        for article in articles:
            link = article['href']
            if "/news/" in link:
                link = f"https://ca.finance.yahoo.com{link}"  # Complete the URL
            if link not in news_links:  # Avoid duplicates
                news_links.append(link)

    return news_links

# Set up WebDriver
# driver.get("https://ca.finance.yahoo.com/quote/MSFT/news/")

# Call the function to scroll and get news links
news_links = natural_scroll_and_parse(driver, pause_time=3, scroll_limit=10, scroll_amount=3000)

# # Print the collected news links
# for link in news_links:
#     print(link)

loop:0
loop:1
loop:2
loop:3
loop:4
loop:5
loop:6
loop:7
loop:8
loop:9


In [9]:
len(news_links)

285

In [82]:
driver.quit()